In [ ]:
# First we importe required libraries and provided them alias
import numpy as np
import pandas as pd
import chardet

In [ ]:
#Importing "companies" data file, provided to us.
companies  = pd.read_csv("companies.txt", sep="\t", encoding = "ISO-8859-1")
#In case any special character available in file, we used following method
companies=companies.applymap(lambda s: s.encode('utf-8').decode('ascii', 'ignore') if type(s) == str else s)
companies

In [ ]:
#Importing "rounds2" data file, provided to us
rounds2  = pd.read_csv("rounds2.csv", encoding="Latin-1")
#In case any special character available in file, we used following method
rounds2=rounds2.applymap(lambda s: s.encode('utf-8').decode('ascii', 'ignore') if type(s) == str else s)
rounds2

In [ ]:
#To match the common attribute between "companies" and "rounds2" file, we made similiar case type.
companies.permalink = companies.permalink.astype(str).str.lower()
companies

In [ ]:
#To match the common attribute between "companies" and "rounds2" file, we made similiar case type.
rounds2.company_permalink = rounds2.company_permalink.astype(str).str.lower()
rounds2

In [ ]:
#unique companies are present in rounds2 
len(sorted(rounds2['company_permalink'].unique()))

In [ ]:
#unique companies are present in companies 
len(set(companies.permalink.tolist())) 

In [ ]:
#Checking there any companies in the rounds2 file which are not  present in companies
print(len(rounds2.loc[~rounds2['company_permalink'].isin(companies['permalink']), :]))

In [ ]:
#Creating Master Dataframe, Cheking output too.
master_frame = pd.merge(rounds2,companies, how='left', left_on=['company_permalink'], right_on=['permalink'])
master_frame

In [ ]:
#Dropping rows having NAN values in "raised_amount_usd" column as it of no use. Created temporary dataframe
master_frame_temp = master_frame[~np.isnan(master_frame['raised_amount_usd'])]

In [ ]:
#Checking "raised_amount_usd" per "funding_round_type"
master_frame_temp.groupby(['funding_round_type']).mean().round(1)

In [ ]:
#Finding top 9 countries.
new_df = master_frame.groupby(['country_code'], sort=True).sum()
top9 = new_df.sort_values(by = ['raised_amount_usd'], ascending=[False]).head(9)
top9

In [ ]:
#Importing "mapping.csv" data file, provided to us.
mapping  = pd.read_csv("mapping.csv", encoding="Latin-1")
#In case any special character available in file, we used following method
mapping=mapping.applymap(lambda s: s.encode('utf-8').decode('ascii', 'ignore') if type(s) == str else s)
mapping.head()

In [ ]:
#There is certain error in mapping file in "category_list" column. "0" is replaced by "na"
mapping.category_list = mapping.category_list.str.replace("0","na")
mapping.head(25)

In [ ]:
#Created "primary_sector" column in master_frame by splitting "category_list"
master_frame['primary_sector']=master_frame['category_list'].str.split('|').str[0]
master_frame.head()

In [ ]:
#Checking percentage of Null in master_frame per column
round(100*(master_frame.isnull().sum()/len(master_frame.index)),2)

In [ ]:
#REMOVING ALL THOSE ROWS WHERE primary_sector IS NULL
master_frame = master_frame[~pd.isnull(master_frame['primary_sector'])]

In [ ]:
#Custom function for mapping main_sector
def mast_cat(arg1):
    if arg1:
        if arg1 in mapping.category_list.tolist():
            df1 = mapping.loc[mapping['category_list'] == arg1]
            df1.set_index('category_list', inplace = True)
            arg2 = (df1 == 1).idxmax(axis=1)[0]
            return arg2
        else:
            return 'Blanks'
    else:
        return 'Blanks' 

In [ ]:
#Adding "main_sector" column and checking output.
master_frame['main_sector'] = master_frame.primary_sector.apply(lambda x: mast_cat(x) )
master_frame

In [ ]:
#Creating D1 satisfying specified conditions.
D1 = master_frame.loc[ (master_frame['country_code'] == 'USA') & (5000000 < master_frame['raised_amount_usd']) & (master_frame['raised_amount_usd'] < 15000000) & (master_frame['funding_round_type'] =='venture') ]
#Adding required columns in D1. Checking output.
D1['count_main'] = D1.groupby('main_sector')['main_sector'].transform('count')
D1['sum_raised_main'] = D1.groupby('main_sector')['raised_amount_usd'].transform('sum')
D1.head()

In [ ]:
#Creating D2 satisfying specified conditions.
D2 = master_frame.loc[ (master_frame['country_code'] == 'GBR') & (5000000 < master_frame['raised_amount_usd']) & (master_frame['raised_amount_usd'] < 15000000) & (master_frame['funding_round_type'] =='venture')]
#Adding required columns in D2. Checking output.
D2['count_main'] = D2.groupby('main_sector')['main_sector'].transform('count')
D2['sum_raised_main'] = D2.groupby('main_sector')['raised_amount_usd'].transform('sum')
D2

In [ ]:
#Creating D3 satisfying specified conditions.
D3 = master_frame.loc[ (master_frame['country_code'] == 'IND') & (5000000 < master_frame['raised_amount_usd']) & (master_frame['raised_amount_usd'] < 15000000) & (master_frame['funding_round_type'] =='venture')]
#Adding required columns in D3. Checking output.
D3['count_main'] = D3.groupby('main_sector')['main_sector'].transform('count')
D3['sum_raised_main'] = D3.groupby('main_sector')['raised_amount_usd'].transform('sum')
D3

In [ ]:
#Finding Total number of Investments (count) for D1
inv_num = D1.drop_duplicates('main_sector')
inv_num.count_main.sum()

In [ ]:
#Finding Total number of Investments (count) for D2
inv_num = D2.drop_duplicates('main_sector')
inv_num.count_main.sum()

In [ ]:
#Finding Total number of Investments (count) for D3
inv_num = D3.drop_duplicates('main_sector')
inv_num.count_main.sum()

In [ ]:
#Finding Total amount of investment (USD) for D1
D1.groupby('main_sector')['sum_raised_main'].unique().sum()

In [ ]:
#Finding Total amount of investment (USD) for D2
D2.groupby('main_sector')['sum_raised_main'].unique().sum()

In [ ]:
#Finding Total amount of investment (USD) for D3
D3.groupby('main_sector')['sum_raised_main'].unique().sum()

In [ ]:
#Finding 3 largest main_sector for D1
D1.drop_duplicates('main_sector').nlargest(3,'count_main')

In [ ]:
#Finding 3 largest main_sector for D2
D2.drop_duplicates('main_sector').nlargest(3,'count_main')

In [ ]:
#Finding 3 largest main_sector for D3
D3.drop_duplicates('main_sector').nlargest(3,'count_main')

In [ ]:
#For top sector count-wise, To find which company received the highest investment ! For D1
find1 = D1.loc[D1['main_sector'] == 'Others']
find1['comp_inv_sum'] = D1.groupby('company_permalink')['raised_amount_usd'].transform('sum')
find1.sort_values('comp_inv_sum', ascending=False).drop_duplicates(['company_permalink'])

In [ ]:
#Forsecond best sector count-wise, which company received the highest investment! For D1
find1 = D1.loc[D1['main_sector'] == 'Social, Finance, Analytics, Advertising']
find1['comp_inv_sum'] = D1.groupby('company_permalink')['raised_amount_usd'].transform('sum')
find1.sort_values('comp_inv_sum', ascending=False).drop_duplicates(['company_permalink'])

In [ ]:
#For top sector count-wise, To find which company received the highest investment ! For D2
find2 = D2.loc[D2['main_sector'] == 'Others']
find2['comp_inv_sum'] = D2.groupby('company_permalink')['raised_amount_usd'].transform('sum')
find2.sort_values('comp_inv_sum', ascending=False).drop_duplicates(['company_permalink'])

In [ ]:
#Forsecond best sector count-wise, which company received the highest investment! For D2
find2 = D2.loc[D2['main_sector'] == 'Cleantech / Semiconductors']
find2['comp_inv_sum'] = D2.groupby('company_permalink')['raised_amount_usd'].transform('sum')
find2.sort_values('comp_inv_sum', ascending=False).drop_duplicates(['company_permalink'])

In [ ]:
#For top sector count-wise, To find which company received the highest investment ! For D3
find3 = D3.loc[D3['main_sector'] == 'Others']
find3['comp_inv_sum'] = D3.groupby('company_permalink')['raised_amount_usd'].transform('sum')
find3.sort_values('comp_inv_sum', ascending=False).drop_duplicates(['company_permalink'])

In [ ]:
#Forsecond best sector count-wise, which company received the highest investment! For D3
find3 = D3.loc[D3['main_sector'] == 'Social, Finance, Analytics, Advertising']
find3['comp_inv_sum'] = D3.groupby('company_permalink')['raised_amount_usd'].transform('sum')
find3.sort_values('comp_inv_sum', ascending=False).drop_duplicates(['company_permalink'])